# Task 5: Model Interpretability

In this task, we use interpretability tools such as SHAP and LIME to explain how the NER model identifies entities in Amharic Telegram messages. This helps ensure transparency, trust, and guides further model improvements.

## Steps
1. Install and import SHAP and LIME
2. Load the fine-tuned NER model
3. Use SHAP and LIME to interpret model predictions
4. Analyze difficult/ambiguous cases
5. Generate interpretability reports and identify areas for improvement

In [ ]:
# Install SHAP and LIME
!pip install shap lime

In [ ]:
import shap
from lime.lime_text import LimeTextExplainer
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# Load the fine-tuned model and tokenizer
MODEL_PATH = '../models/fine_tuned_ner_model'  # Update path as needed
model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model.eval()

In [ ]:
# Define a prediction function for LIME/SHAP

def predict_proba(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs).logits
    # For demonstration, return softmax over the first token (can be adapted for sequence labeling)
    probs = torch.nn.functional.softmax(outputs, dim=-1)
    return probs[:, 0, :].numpy()


In [ ]:
# LIME explanation for a sample message
explainer = LimeTextExplainer(class_names=list(model.config.id2label.values()))
sample_text = "አዲስ ምርት ዋጋ 1000 ብር ቦሌ ሱቅ"
exp = explainer.explain_instance(sample_text, predict_proba, num_features=10)
exp.show_in_notebook(text=sample_text)

In [ ]:
# SHAP explanation for a sample message
explainer = shap.Explainer(predict_proba, tokenizer)
shap_values = explainer([sample_text])
shap.plots.text(shap_values)

## Analysis of Difficult Cases and Summary

- Review LIME and SHAP explanations for ambiguous or overlapping entities.
- Identify patterns where the model struggles (e.g., similar product and location names, price ambiguity).
- Use interpretability insights to suggest improvements (e.g., more training data, better tokenization, or model architecture adjustments).

This interpretability analysis increases trust in the NER system and guides future enhancements.